# Modelo Preditivo de Magnitude de Terremotos
Este notebook treina um modelo de machine learning para prever a magnitude de um terremoto com base em profundidade, localização geográfica e placa tectônica.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import numpy as np


## Carregamento dos dados
Aqui lemos os dados processados e criamos uma coluna de exemplo chamada `placa_tectonica` caso ela ainda não exista, para simular esse fator geológico.

In [2]:
df = pd.read_csv("../dados_processados/terremotos_processados.csv")

# Criar coluna simulada
if "placa_tectonica" not in df.columns:
    placas = ['Pacífica', 'Nazca', 'Cocos', 'Eurasiática', 'Sul-Americana']
    df['placa_tectonica'] = np.random.choice(placas, size=len(df))

df.head()

,data_hora,local,magnitude,profundidade_km,longitude,latitude,nivel,placa_tectonica
0,2024-12-30 05:49:02.808,"13 km NW of Port-Vila, Vanuatu",5.1,66.612,168.2183,-17.6555,leve,Eurasiática
1,2024-12-30 05:41:06.678,"63 km W of Coquimbo, Chile",5.5,10.000,-71.9959,-29.9272,leve,Nazca
2,2024-12-30 05:40:49.297,"68 km W of Coquimbo, Chile",5.5,10.000,-72.0531,-29.9251,leve,Nazca
3,2024-12-30 02:56:16.807,"18 km SE of Bangui, Philippines",5.5,34.000,120.9038,18.4391,leve,Sul-Americana
4,2024-12-29 22:20:41.330,"34 km NNW of Āwash, Ethiopia",5.1,10.000,40.0491,9.2754,leve,Cocos


## Construção do Modelo
Usamos um `RandomForestRegressor`, um modelo baseado em várias árvores de decisão, ideal para prever valores contínuos como a magnitude.

In [3]:
X = df[["profundidade_km", "latitude", "longitude", "placa_tectonica"]]
y = df["magnitude"]

categorical_features = ["placa_tectonica"]
numeric_features = ["profundidade_km", "latitude", "longitude"]

preprocessor = ColumnTransformer([
    ("onehot", OneHotEncoder(), categorical_features)
], remainder='passthrough')

pipeline = Pipeline([
    ("preprocessamento", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])


## Treinamento e Avaliação
Dividimos os dados entre treino e teste, treinamos o modelo e avaliamos com métricas como MAE, RMSE e R². 
- MAE (erro médio absoluto)
- RMSE (raiz do erro quadrático médio)
- R² (coeficiente de determinação — quanto o modelo explica dos dados)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Avaliação do Modelo:")
print(f"MAE:  {mean_absolute_error(y_test, y_pred):.3f}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.3f}")
print(f"R²:   {r2_score(y_test, y_pred):.3f}")


Avaliação do Modelo:
MAE:  0.286
RMSE: 0.407
R²:   0.014


## Salvamento do Modelo
O modelo é salvo com `joblib` para ser reutilizado sem precisar treinar novamente.

In [5]:
joblib.dump(pipeline, "../modelo_magnitude_terremoto.pkl")


['../modelo_magnitude_terremoto.pkl']